In [7]:
import numpy as np
import tensorflow as tf 
import tensorflow_datasets as tfds
import tensorflow_hub as hub


In [13]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]
Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 1 examples [00:02,  2.76s/ examples]
Generating train examples...: 1979 examples [00:03, 674.92 examples/s]
Generating train examples...: 3977 examples [00:04, 1108.29 examples/s]
Generating train examples...: 5910 examples [00:05, 1370.98 examples/s]
Generating train examples...: 7750 examples [00:06, 1517.28 examples/s]
Generating train examples...: 9665 examples [00:07, 1639.64 examples/s]
Generating train examples...: 11537 examples [00:08, 1710.54 examples/s]
Generating train examples...: 13553 examples [00:09, 1803.21 examples/s]
Generating train examples...: 15440 examples [00:10, 1793.02 examples/s]
Generating train examples...: 17291 examples [00:11, 1761.89 examples/s]
Generating train examples...: 19094 examples [00:12, 1750.53 examples/s]
Generating train examples...: 21332 examp

Dataset imdb_reviews downloaded and prepared to C:\Users\HP\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [14]:
train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [15]:
train_example_batch,train_labels_batch = next(iter(train_data.batch(10)))

In [16]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [17]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [22]:
#emmbedding
embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer=hub.KerasLayer(embedding, input_shape= [],dtype=tf.string, trainable=True)

In [23]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [28]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_4 (Dense)             (None, 16)                336       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [33]:
history = model.fit(train_data.shuffle(10000).batch(100),epochs=25,validation_data=validation_data.batch(100),verbose=1)

Epoch 1/25
150/150 [==============================] - 2s 16ms/step - loss: 0.2845 - accuracy: 0.8818 - val_loss: 0.3302 - val_accuracy: 0.8410
Epoch 2/25
150/150 [==============================] - 3s 17ms/step - loss: 0.2278 - accuracy: 0.9073 - val_loss: 0.3071 - val_accuracy: 0.8661
Epoch 3/25
150/150 [==============================] - 3s 22ms/step - loss: 0.1918 - accuracy: 0.9257 - val_loss: 0.3036 - val_accuracy: 0.8719
Epoch 4/25
150/150 [==============================] - 4s 25ms/step - loss: 0.1641 - accuracy: 0.9406 - val_loss: 0.3113 - val_accuracy: 0.8608
Epoch 5/25
150/150 [==============================] - 4s 29ms/step - loss: 0.1417 - accuracy: 0.9494 - val_loss: 0.3089 - val_accuracy: 0.8737
Epoch 6/25
150/150 [==============================] - 3s 22ms/step - loss: 0.1225 - accuracy: 0.9593 - val_loss: 0.3168 - val_accuracy: 0.8717
Epoch 7/25
150/150 [==============================] - 4s 23ms/step - loss: 0.1051 - accuracy: 0.9659 - val_loss: 0.3295 - val_accuracy: 0.8679

In [35]:
results = model.evaluate(test_data.batch(500), verbose=2)
for name, value in zip(model.metrics_names, results):print("%s: %.3f" % (name, value))

50/50 - 2s - loss: 0.7816 - accuracy: 0.8365 - 2s/epoch - 32ms/step
loss: 0.782
accuracy: 0.836
